<h1 style="display:inline-block">Iot Essay</h1><h2>Sample Python code to implement IoT</h2>

This is a code used in a jupyter notebook that is a step-by-step on how to implement an Iot network using GCP.

In [ ]:

# funcao que cria o token de autenticacao com a rede
def create_jwt( project_id, private_key_fname ):
    
    try:
        import datetime

        token = {
            # o momento em que o token foi criado
            'iat': datetime.datetime.utcnow(),

            # momento em que o token expira. Em geral nao devemos usar intervalos tao grandes
            # estamos usando meia hora para fins deste exercico
            'exp': datetime.datetime.utcnow() + 
                datetime.timedelta( minutes=30 ),

            # no campo "audiencia" colocamos o project_id
            'aud': project_id
        }

        # le o arquivo com a chave privada
        from google.cloud import storage
        
        storage_client = storage.Client()
        
            # note que estamos utilizando um bucket hardcoded com o nome do projeto
        print( "project_id " + project_id )
        bucket = storage_client.get_bucket( project_id )
        
        blob = bucket.blob( private_key_fname )
        
        private_key = blob.download_as_string().decode( "utf-8" )
    
        import jwt

        # criptografa o token. Note que temos que indicar o tipo de chave 
        # (algoritmo de criptografia utilizado na assinatura)
        return jwt.encode( token, private_key, algorithm='RS256' )

    except Exception as error:
        
        import logging
        logging.error( error )
        return None
    

connected = 255 # maior que zero = conexao recusada

def GPS_data( request ):
    print( "entrando na Cloud Function" )
    global connected
    
    import logging
    
    try:
        msg = request.get_json( force=True )
        #msg = request
        
        data = msg[ "message" ][ "data" ]

        import json
        import pybase64

        data_b64 = pybase64.b64decode( data )
        inner = json.loads( data_b64 )

        latitude = inner[ "latitude" ]
        longitude = inner[ "longitude" ]
        ts = inner[ "timestamp" ]

        # id do cliente MQTT identifica o dispositivo na nuvem
        import os
            # utilizando o mesmo projeto e região da cloud function
        project_id = os.environ.get( 'GCP_PROJECT', '' ) 
        
        if len( project_id ) < 4:
            project_id = os.popen( 'gcloud config get-value project' ).read().strip()
        
        cloud_region = os.environ.get( 'FUNCTION_REGION', "" )

        if len( cloud_region ) < 4:
            cloud_region = 'us-central1'

        registry_id = "taxi-iot-broker"

        # vamos usar o nome do projeto como id do dispositivo. Na realidade teriamos nomes exclusivos 
        # para cada dispositivo na rede
        device_id = project_id

        client_id = 'projects/{}/locations/{}/registries/{}/devices/{}'.format(
                project_id, cloud_region, registry_id, device_id )

        import paho.mqtt.client as mqtt

        client = mqtt.Client( client_id=client_id )

        # No Google Cloud IoT Core o campo username é ignorado, e o campo
        # password é usado para transmitir um JWT para autorizar o dispositivo
        client.username_pw_set( username='unused', password=create_jwt( project_id, "rsa_private.pem" ))

        # Ative o suporte a SSL/TLS
        client.tls_set( ca_certs=None, certfile=None, keyfile=None, ciphers=None )

        # a conexao é assincrona, um evento de on_connect sera devolvido quando a conexao
        # ocorrer
        def mqtt_connect( client, userdata, flags, rc ):
            global connected
            connected = rc

        client.on_connect=mqtt_connect 

        # conecta com a ponte de IoT do GCP
        # o bridge de IoT do GCP é o ponto de entrada na nuvem - endereço padrão do IoT Core
        mqtt_bridge_hostname = "mqtt.googleapis.com"
        mqtt_bridge_port = 8883

        client.connect( mqtt_bridge_hostname, mqtt_bridge_port )

        # a conexao é assincrona, temos q esperar em loop até ela ocorrer
        # ou até um timeout
        client.loop_start() 

        import time
        import datetime

        print( datetime.datetime.now().time())
        
        t_end = time.time() + 60 
        
        while time.time() < t_end and connected > 0:
            time.sleep( 1 )

        print( datetime.datetime.now().time())

        print( "conexao MQTT -> status " + str( connected ))

        # Topicos padrao para envio de mensagens: events e state
        # note que nao estamos checando connected == 0 (q indica conexao com sucesso )
        # o que quer dizer q se a conexao falhar por algum motivo (internet caiu, p ex)
        # a mensagem nao vai ser enviada (mas nao vai levantar erro)
        mqtt_topic = '/devices/{}/events'.format( device_id )

            # qos (quality of service) = 1 indica entrega pelo menos uma vez 
            # (mesma qualidade do Pub/Sub)

        payload = { "ts": ts, "latitude": latitude, "longitude": longitude }
        client.publish( mqtt_topic, json.dumps( payload ), qos=1 )

        print( "publicado payload " )

        client.loop_stop()    #Stop loop 
        client.disconnect() # disconnect

    except Exception as error:
        import logging
        logging.error( error )
        raise

        
#GPS_data( {"message": { "data": "eyJyaWRlX2lkIjoiZjA4MTQ1OTAtY2FiMi00OGMyLWIzZjItYTc5MzA4ODdlOTlhIiwicG9pbnRfaWR4IjozMCwibGF0aXR1ZGUiOjQwLjc2MjYwMDAwMDAwMDAwNiwibG9uZ2l0dWRlIjotNzMuOTc1MTcsInRpbWVzdGFtcCI6IjIwMjAtMDYtMThUMTg6Mjk6MTcuNjgwMjEtMDQ6MDAiLCJtZXRlcl9yZWFkaW5nIjoxLjM0ODYyMzksIm1ldGVyX2luY3JlbWVudCI6MC4wNDQ5NTQxMywicmlkZV9zdGF0dXMiOiJlbnJvdXRlIiwicGFzc2VuZ2VyX2NvdW50IjoyfQ=="}} )
